In [19]:

import fitz
import camelot
import warnings , math, collections
import pickle
from collections import Counter

warnings.filterwarnings("ignore", category=UserWarning) 

#path = r"C:\Users\rando\OneDrive\Documents\mywork-repo"
path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo"


pickle_path_text = r"\output\pkl\textual_data.pkl"
pickle_path_tab = r"\output\pkl\tabular_data.pkl"
samco_path = r"\files\SamcoFactSheet2024.pdf"
tata_path = r"\files\TataFactSheet2024.pdf"
dry_run_path = r"\output\DryRun.pdf"

In [ ]:
"""Basic page cleaning, image removal, page number removal,
repetitive data removal"""

In [44]:
with open(path + pickle_path_text , 'rb') as file:
    textual_data = pickle.load(file)

In [42]:
#aggregate code
from collections import Counter

def get_pdf_aggregates(block_data):
    pdf_sizes = []
    pdf_colors = []
    pdf_words = []
    
    document = block_data['blocks']
    for page in document:
        page_size, page_color, page_text = [], [], []
        for block in page:
            for line in block['lines']:
                for span in line['spans']:
                    page_color.append(span['color'])
                    page_size.append(span['size'])
                    page_text.append(span['text'])
        pdf_colors.append(page_color)
        pdf_sizes.append(page_size)
        pdf_words.append(page_text)

    grand_pdf_colors = Counter([color for sublist in pdf_colors for color in sublist])
    grand_pdf_sizes = Counter([size for sublist in pdf_sizes for size in sublist])
    grand_pdf_words = Counter([word for sublist in pdf_words for word in sublist])

    return {
        "grand": {
            "text": grand_pdf_words,
            "color": grand_pdf_colors,
            "size": grand_pdf_sizes
        },
        "page": {
            'text': [Counter(page) for page in pdf_words],
            'size': [Counter(page) for page in pdf_sizes],
            'color': [Counter(page) for page in pdf_colors]
        }
    }



data = get_pdf_aggregates(textual_data)

In [80]:
def get_data_from_page(data:list, page:int)->list:
    
    block_data = data['blocks']
    return block_data[page-1]

def print_page(data:list, output_path:str):
    new_doc = fitz.open(output_path)
    


page_data = get_data_from_page(textual_data, 4)
page_data.sort(key= lambda k: (k['bbox'][1], k['bbox'][0]))

In [81]:
def process_page_blocks(page_data):
    
    text_color_size = []
    for block in page_data:
        bbox = block['bbox']  # Bounding box of the block
        block_text = []  # List to collect text from all spans in this block
        block_size = []
        block_color = []
        for line in block['lines']:
            for span in line['spans']:
                block_text.append(span['text'])
                block_size.append(span['size'])
                block_color.append(span['color'])
        stitched_text = '\t'.join(block_text)  # Join all text into a single string
        print(f"BBox: {bbox}, text: {stitched_text} {block_color}, {block_size}")
        text_color_size.append((stitched_text, block_color[0], block_size[0]))
        
    return text_color_size

# Example usage with your structured data
process_page_blocks(page_data)

BBox: (36.273399353027344, 67.16098022460938, 402.72222900390625, 101.55298614501953), text: Samco Active Momentum Fund [-12684359], [24.0]
BBox: (43.30080032348633, 121.75519561767578, 136.52279663085938, 134.5802001953125), text: Investment Objective [-1], [9.0]
BBox: (541.001953125, 132.4703826904297, 573.7257080078125, 151.66432189941406), text: % to net	exposure of	Net Assets [-12220216, -12220216, -12220216], [5.848076343536377, 5.848076343536377, 5.848076343536377]
BBox: (43.3916015625, 143.67257690429688, 535.677490234375, 229.14979553222656), text: -20.22%	The investment objective of the Scheme is to seek to 	generate long-term capital appreciation by investing in 	stocks showing strong momentum. Momentum stocks are 	such that exhibit positive price momentum – based on the 	phenomenon that stocks which have performed well in the 	past relative to other stocks (winners) continue to perform 	well in the future, and stocks that have performed 	relatively poorly (losers) continue 